# Продвинутый Python, лекция 13

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Коган Александра, Романченко Полина

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

## Базы данных

Вначале немного подушним (но полезно подушним). Есть два понятия: БД (база данных) и СУБД (система управления базы данных). В чем разница?

* БД - набор данных (таблиц), логически связанных между собой

* СУБД - набор инструментов, которые позволяют что-то делать с БД

Какие СУБД бывают? Реляционные и нереляционные

* Реляционные - нужна структура, нормальные формы etc (MySQL, PostgreSQL)

* Нереляционные - можно хранить что угодно и примерно как угодно (MongoDB)

Мы будем говорить про реляционные (потому что это стандарт, SQL, различаются между СУБД) 

## Как подключаться к БД?

Первое, что нам нужно при любой работе - это уметь подключаться к базе данных. А как, а что?

База данных обычно лежит на отдельном сервере, где и крутится сама по себе (что-то обновляет etc). А приложение на другом сервере к нему подключается. Поэтому надо уметь подключаться :з 

Подключение идет через т.н. драйвера, но люди уже давно этим занимаются, поэтому подключение происходит через менеджер драйверов, который позволяет легко подключиться практически к любой СУБД:

* ODBC (Open Database Connectivity)

```
Driver=(SQL Server);Server=(serverName);Database=(instanceName)(;property=value)
```

* JDBC (Java Database Connectivity)

```
Driver=(SQL Server);Server=(serverName);Database=(instanceName)(;property=value)
```

* Кастомные драйвера

В общем виде что должно быть при подключении:

- тип СУБД

- хост

- порт

- название БД

- данные для авторизации (логин-пароль)

## Теперь к Python

In [ ]:
!pip install pyodbc #ODBC
!pip install jaydebeapi #JDBC
!pip install psycopg2 #PostgreSQL (отдельно, не мучаясь с менеджерами)

Все коннекторы к БД внутри Python должны подчиняться [pep249](https://peps.python.org/pep-0249/) - спецификация для бибилотек с БД. Как выглядит абсолютно любая библиотека:

![](https://i.pinimg.com/originals/be/c7/c1/bec7c10b61f5cb2b851d04f0fbeeee2e.png)

Метод connect - возращает объект типа Connection (связь с БД непосредственно), который должен обладать методами:

* cursor() - получить объект типа cursor (для доступа к данным)

* commit(), rollback() - необходимо для транзакций

* close() - закрыть связь (всегда надо, потому что держит ресурсы)

Разберем это все на базовом СУБД - sqlite3 (есть всегда)

In [ ]:
import sqlite3

connection = sqlite3.connect("chinook.db") # открыли
connection.close() # закрыли

![](https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)

Что такое cursor?

Cursor - это объект для выполнения запросов и получения результатов (по сути объект контеста для запроса, а мы где вообще)

И cursor тоже стоит закрывать (но вообще он закрывается вместе с connection) :)

In [ ]:
connection = sqlite3.connect("chinook.db") # открыли
cursor = connection.cursor()
set(dir(cursor)) - set(dir(object)) #смотрим на методы
# execute - выполни запрос
# fetch - дай результаты

{'__iter__',
 '__next__',
 'arraysize',
 'close',
 'connection',
 'description',
 'execute',
 'executemany',
 'executescript',
 'fetchall',
 'fetchmany',
 'fetchone',
 'lastrowid',
 'row_factory',
 'rowcount',
 'setinputsizes',
 'setoutputsize'}

In [ ]:
cursor.execute("SELECT name, type, rootpage FROM sqlite_master WHERE type='table'")

In [ ]:
one_row = cursor.fetchone() # дай одну строку, 1 строка - 1 кортеж
some_rows = cursor.fetchmany(5) # дай много строк
remain_rows = cursor.fetchall() # дай все остальные строки
print(one_row)
print('-' * 30)
print(some_rows)
print('-' * 30)
print(remain_rows)
print('-' * 30)
# Курсор буквально ходит по таблице
print(cursor.fetchall())

('albums', 'table', 2)
------------------------------
[('sqlite_sequence', 'table', 3), ('artists', 'table', 4), ('customers', 'table', 5), ('employees', 'table', 8), ('genres', 'table', 10)]
------------------------------
[('invoices', 'table', 11), ('invoice_items', 'table', 13), ('media_types', 'table', 15), ('playlists', 'table', 16), ('playlist_track', 'table', 17), ('tracks', 'table', 20), ('sqlite_stat1', 'table', 864)]
------------------------------
[]


## Что умеем делать?

Надобно делать запросы, и получать ответ. А как писать запросы? Есть несколько вариантов:

* Чисто с помощью SQL - кайф, но не очень удобно (мы же в Питоне)

* Query Builder - удобнее, но нужна отдельная библиотека

* ORM - уровень посложнее

### Часть 1. Сырые запросы

In [ ]:
def search_by_name(name, cursor):
    cursor.execute(f"SELECT * FROM playlists WHERE Name LIKE '%{name}%'")
    return cursor.fetchall()

print(search_by_name("Brazil", cursor))

[(11, 'Brazilian Music')]


Проблема - легко ломается! (можно передать какую-нибудь дичь, SQL injection etc). Давайте делать все более безопасно - через словари:

In [ ]:
cursor.execute("SELECT * FROM playlists WHERE Name LIKE :name", {"name": f'%{name}%'})
# двоеточие - название переменных из словаря, драйвер сам все сделает, а также делает экранирование (и преобразование типов)

Но дополнительная сила делать все через словари - это возможность сделать сразу несколько запросов в одном! (например, для вставки значений)

In [ ]:
cursor.executemany("""
INSERT INTO tracks (Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice)
VALUES(?, ?, ?, ?, ?, ?, ?, ?)
""", [
    ("name", 1, 1, 1, None, 1, 0, 0),
    ("name_2", 2, 1, 1, None, 1, 0, 0)
])
cursor.execute("SELECT * FROM tracks WHERE Name LIKE 'name%'")
cursor.fetchall()

[(3504, 'name', 1, 1, 1, None, 1, 0, 0),
 (3505, 'name_2', 2, 1, 1, None, 1, 0, 0)]

### Транзакции

Посмотрим на вот этот код:

In [ ]:
connection_2 = sqlite3.connect("chinook.db") # открыли
cursor_2 = connection_2.cursor()
cursor_2.execute("SELECT * FROM tracks WHERE Name LIKE 'name%'")
cursor_2.fetchall()

[]

Вообще ничего нет. А где...

Если посмотреть в файлы ноутбука, то мы увидим такую вещь, как chinook.db-journal. Это история всех изменений. Мы их сделали, но не закоммитили! А чтобы это сделать, надо сделать commit()

In [ ]:
connection.commit() # сделать commit
# connection.rollback() # отменить все это добро, до commit() надо делать, так как после commit не откатить

In [ ]:
cursor_2.execute("SELECT * FROM tracks WHERE Name LIKE 'name%'")
cursor_2.fetchall()

[(3504, 'name', 1, 1, 1, None, 1, 0, 0),
 (3505, 'name_2', 2, 1, 1, None, 1, 0, 0)]

In [ ]:
connection.close()
connection_2.close()

### Часть 2. Query Builder

В чем плюсы?

1. Вы не пишите SQL код

2. Делаете по частям (логическим), форматировать не надо

3. Можно с разными диалектами (иногда может в ногу выстрелить)

4. Чуть менее гибкий (но редко когда на это наткнешься, не видел я таких)

Но запрос руками, обработка руками (все равно ручками много что делать). Это все библиотека [PyPika](https://pypika.readthedocs.io/en/latest/) (а есть еще другие, но по сути, это одно и то же, вообще в теории и руками можно написать)

In [ ]:
!pip install pypika

#### Как формируются запросы?

In [ ]:
from pypika import Query

q = Query.from_('tracks').select("TrackId", "Name").limit(10) #создаем запрос, после from_ можно вообще в любом порядке писать
q_1 = Query.from_('tracks').limit(10).select("TrackId", "Name")
str(q), str(q_1) # Это все просто текст, запроса к БД еще нет

('SELECT "TrackId","Name" FROM "tracks" LIMIT 10',
 'SELECT "TrackId","Name" FROM "tracks" LIMIT 10')

#### Подбор под диалекты:

In [ ]:
from pypika import MSSQLQuery, OracleQuery, PostgreSQLQuery

oracle = OracleQuery.from_('tracks').select("TrackId", "Name").limit(10)
ms = MSSQLQuery.from_('tracks').select("TrackId", "Name").limit(10) # в MSSQL не существует LIMIT))0)
postgre = PostgreSQLQuery.from_('tracks').select("TrackId", "Name").limit(10)
str(oracle), str(ms), str(postgre)

('SELECT TrackId,Name FROM tracks LIMIT 10',
 'SELECT "TrackId","Name" FROM "tracks" OFFSET 0 ROWS FETCH NEXT 10 ROWS ONLY',
 'SELECT "TrackId","Name" FROM "tracks" LIMIT 10')

#### Давайте усложнять и упрощать!

In [ ]:
from pypika import Table, Field # более явный способ разметки

tracks = Table("tracks")
q = Query.from_(tracks) \
    .where(Field('Name').like('%Music%')).select("TrackId", 'Name') \
    .where(Field('TrackId') < 100)

print(str(q))

q = Query.from_(tracks) \
    .where(Field('Name').like('\' OR 1=1')).where(tracks.TrackId < 100) \
    .select('TrackId', 'Name') #прикол экранирования и SQL-injection

print(str(q))

SELECT "TrackId","Name" FROM "tracks" WHERE "Name" LIKE '%Music%' AND "TrackId"<100
SELECT "TrackId","Name" FROM "tracks" WHERE "Name" LIKE ''' OR 1=1' AND "TrackId"<100


#### Ну и параметры:

In [ ]:
from pypika import Parameter

q = Query.from_(tracks) \
    .where(Field('Name').like(Parameter(':name'))).where(tracks.TrackId < 100) \
    .select('TrackId', 'Name') #если не делать параметр, то он посчитает это просто строкой

print(str(q))

SELECT "TrackId","Name" FROM "tracks" WHERE "Name" LIKE :name AND "TrackId"<100


#### Далее у нас наши любимые JOINы:

In [ ]:
albums = Table('albums')

q = q.inner_join(albums)
q = q.using('AlbumId') #второй вариант, суть такая же
#q = q.on(tracks.AlbumId == albums.AlbumId) # Первый вариант
q = q.select(albums.Title)
str(q)

'SELECT "tracks"."TrackId","tracks"."Name","albums"."Title","albums"."Title" FROM "tracks" JOIN "albums" ON "tracks"."AlbumId"="albums"."AlbumId" JOIN "albums" USING ("AlbumId") WHERE "Name" LIKE :name AND "tracks"."TrackId"<100'

#### Подзапросы:

In [ ]:
import pypika.functions as fn
from pypika import Order

q1 = Query.from_(tracks).groupby('GenreId').select('GenreId', fn.Count('*').as_('num_tracks'))
print(str(q1))

num_tracks = Field('num_tracks')
q2 = Query.from_('genres').join(q1).using('GenreId').select('Name', num_tracks).orderby(num_tracks, order=Order.desc).limit(10)

print(str(q2))

SELECT "GenreId",COUNT(*) "num_tracks" FROM "tracks" GROUP BY "GenreId"
SELECT "genres"."Name","num_tracks" FROM "genres" JOIN (SELECT "GenreId",COUNT(*) "num_tracks" FROM "tracks" GROUP BY "GenreId") "sq0" USING ("GenreId") ORDER BY "num_tracks" DESC LIMIT 10


#### А теперь в Sqlite3

In [ ]:
connection = sqlite3.connect('chinook.db')

def get_res(query):
    cursor = connection.cursor()
    cursor.execute(str(query))
    cols = [full_column_info[0] for full_column_info in cursor.description]
    res = cursor.fetchall()
    return [{name: value for name, value in zip(cols, r)} for r in res]

get_res(q2)

[{'Name': 'Rock', 'num_tracks': 1299},
 {'Name': 'Latin', 'num_tracks': 579},
 {'Name': 'Metal', 'num_tracks': 374},
 {'Name': 'Alternative & Punk', 'num_tracks': 332},
 {'Name': 'Jazz', 'num_tracks': 130},
 {'Name': 'TV Shows', 'num_tracks': 93},
 {'Name': 'Blues', 'num_tracks': 81},
 {'Name': 'Classical', 'num_tracks': 74},
 {'Name': 'Drama', 'num_tracks': 64},
 {'Name': 'R&B/Soul', 'num_tracks': 61}]

### Часть 3. ORM

Ну и теперь вершина - ORM (Object-Relational Mapping)

Плюсы:

* Еще удобнее, чем Query Builder (вообще SQL не почувствуем)

* Таблицы - почти DataClass

* Результат автоматически отображается в красивые объекты

Минусы:

* Иногда нафиг не надо (бывает хуже производительность)

Пример: [SQLAlchemy](https://www.sqlalchemy.org/)

Выглядит это вот так:

![](http://aosabook.org/images/sqlalchemy/layers.png)

* Python core - все, что про cursor, connection etc

* Core - это похожее на Query Builder структура, возможность создавать запросы

* ORM - уже модуль для маппинга результатов к объектам

#### Движок

In [ ]:
!pip install sqlalchemy

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base

# Единственное место с подсоединением к СУБД
engine = create_engine('sqlite+pysqlite:///chinook.db', echo=True)
Base = declarative_base() # предок для всех моделей (таблиц)

#### Модели

In [ ]:
class Tracks(Base):
    __tablename__ = 'tracks' # имя таблицы

    track_id = Column(Integer, name='TrackId', primary_key=True) # обязательно должен быть PrimaryKey
    Name = Column(String)
    GenreId = Column(Integer)
    UnitPrice = Column(Float)
    MediaTypeId = Column(Integer)
    Milliseconds = Column(Integer)

    # создаем колонки, дублируя БД

Ничего не напоминает?)

#### Сессия

Что такое Session? По сути дела Connection в другом виде, содержит в себе объекты моделей, ленивое подключение (когда надо)

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine, future=True, expire_on_commit=False) #future - использование второй версии, expire_on_commit - убиваем на commitу

session = Session() #объект сессии
session_2 = Session()

#### Select

In [ ]:
from sqlalchemy import select

query = select(Tracks).where(Tracks.Name.like('%name%'))

retr_1 = session.execute(query).scalar() #scalar - преобразование результата в объект
retr_2 = session.execute(query).scalar() 

retr_3 = session.execute(select(Tracks).where(Tracks.Name.like('%name%'))).scalar()

res = retr_1 is retr_2, retr_1 is retr_3, retr_2 is retr_3

res

Вообще запросы такие же, как и в Query Builder

#### Insert

In [ ]:
new_track = Tracks(track_id=100000, MediaTypeId=1, Name='new_track', GenreId=1, UnitPrice=1.0, Milliseconds=15)
session.add(new_track)
# ничего не сделал...

In [ ]:
res = session.execute(select(Tracks).filter_by(Name='new_track')).scalar() #а вот тут случился insert
res is new_track

2022-12-04 15:08:41,458 INFO sqlalchemy.engine.Engine INSERT INTO tracks ("TrackId", "Name", "GenreId", "UnitPrice", "MediaTypeId", "Milliseconds") VALUES (?, ?, ?, ?, ?, ?)


INFO:sqlalchemy.engine.Engine:INSERT INTO tracks ("TrackId", "Name", "GenreId", "UnitPrice", "MediaTypeId", "Milliseconds") VALUES (?, ?, ?, ?, ?, ?)


2022-12-04 15:08:41,464 INFO sqlalchemy.engine.Engine [generated in 0.00659s] (100000, 'new_track', 1, 1.0, 1, 15)


INFO:sqlalchemy.engine.Engine:[generated in 0.00659s] (100000, 'new_track', 1, 1.0, 1, 15)


2022-12-04 15:08:41,470 INFO sqlalchemy.engine.Engine SELECT tracks."TrackId", tracks."Name", tracks."GenreId", tracks."UnitPrice", tracks."MediaTypeId", tracks."Milliseconds" 
FROM tracks 
WHERE tracks."Name" = ?


INFO:sqlalchemy.engine.Engine:SELECT tracks."TrackId", tracks."Name", tracks."GenreId", tracks."UnitPrice", tracks."MediaTypeId", tracks."Milliseconds" 
FROM tracks 
WHERE tracks."Name" = ?


2022-12-04 15:08:41,474 INFO sqlalchemy.engine.Engine [generated in 0.00414s] ('new_track',)


INFO:sqlalchemy.engine.Engine:[generated in 0.00414s] ('new_track',)


True

#### Relations (или же Joins)

In [ ]:
from sqlalchemy import ForeignKey, select
from sqlalchemy.orm import relationship

class RelatedTrack(Base):
    __tablename__ = 'tracks'
    __table_args__ = {'extend_existing': True} #дабы не было конфликтов (то есть ссылка есть всегда)


    track_id = Column(Integer, name='TrackId', primary_key=True) # обязательно должен быть PrimaryKey
    Name = Column(String)
    GenreId = Column(Integer, ForeignKey('genres.GenreId'))
    UnitPrice = Column(Float)
    MediaTypeId = Column(Integer)
    Milliseconds = Column(Integer)

    genre = relationship('Genre', back_populates='tracks')

class Genre(Base):
    __tablename__ = 'genres'
    __table_args__ = {'extend_existing': True}

    GenreId = Column(Integer, primary_key=True)
    Name = Column(String)

    tracks = relationship("RelatedTrack", back_populates='genre', uselist=True) #uselist - список

In [ ]:
genre_1 = session.execute(select(Genre).filter_by(GenreId=1)).scalar()
len(genre_1.tracks)

In [ ]:
genre_1.tracks[0].Name

'For Those About To Rock (We Salute You)'

In [ ]:
genre_1.tracks[0].genre is genre_1 #есть обратная связь

True

#### Фильтрация по связям

In [ ]:
tracks = session.execute(select(RelatedTrack).join(Genre).where(Genre.Name.like('%Rock%')).limit(5)).scalars().all()
for track in tracks:
    print(track.genre.Name)

## Животное дня

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/Wied%27s_Marmoset_at_Blank_Park_Zoo.gk.jpg/1920px-Wied%27s_Marmoset_at_Blank_Park_Zoo.gk.jpg)

Это игрунка Жоффруа. Меня всегда поражало, что морда очень похожа на человеческое лицо. А еще у нее есть симпатричный вид (то есть животное, которое живет в одном ареале) - это Золотистоголовая львиная игрунка 

![](https://upload.wikimedia.org/wikipedia/commons/c/c8/Tamarin_Tête_de_lion.jpg)

Они часто путешествуют вместе с группами игрунок Куля, ищет с ними пищу и предупреждает о нападении хищников. Социальное поведение у игрунок Куля сильно развито, большую часть дня животные проводят в совместных поисках пищи, взаимной чистке и играх. О молодняке проявляют заботу все члены группы. Повзрослев, самки остаются в родной группе, а самцы покидают её. (у них тоже матриархат)